<a href="https://colab.research.google.com/github/sriramprasad15/Grammatical-Acceptability-Classifier/blob/main/Grammatical_Acceptability_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load pretrained tokenizer and model trained on CoLA
model_name = "textattack/bert-base-uncased-CoLA"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Set model to evaluation mode
model.eval()

# User input
while True:
    sentence = input("\nEnter an English sentence (or 'exit' to quit): ")
    if sentence.lower() == "exit":
        break

    # Tokenize and encode
    inputs = tokenizer(sentence, return_tensors="pt")

    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Convert logits to probabilities
    probs = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs).item()
    confidence = probs[0][predicted_class].item()

    # CoLA labels: 0 = unacceptable, 1 = acceptable
    if predicted_class == 1:
        print(f"✔️ Acceptable sentence (Confidence: {confidence:.2f})")
    else:
        print(f"❌ Not acceptable (ungrammatical) sentence (Confidence: {confidence:.2f})")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]


Enter an English sentence (or 'exit' to quit): hello this is ram
✔️ Acceptable sentence (Confidence: 0.77)

Enter an English sentence (or 'exit' to quit): am i zero
✔️ Acceptable sentence (Confidence: 0.94)

Enter an English sentence (or 'exit' to quit): no there fun
❌ Not acceptable (ungrammatical) sentence (Confidence: 0.94)

Enter an English sentence (or 'exit' to quit): exit


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from textblob import TextBlob
import torch
import torch.nn.functional as F

# Load model and tokenizer
model_name = "textattack/bert-base-uncased-CoLA"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

def get_bert_prediction(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    probs = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs).item()
    confidence = probs[0][predicted_class].item()
    return predicted_class, confidence

def get_textblob_correction(sentence):
    blob = TextBlob(sentence)
    corrected = str(blob.correct())
    return corrected

# Loop to test multiple sentences
while True:
    sentence = input("\nEnter an English sentence (or 'exit' to quit): ")
    if sentence.lower() == "exit":
        break

    # BERT prediction
    pred_class, confidence = get_bert_prediction(sentence)
    bert_result = "✔️ Acceptable" if pred_class == 1 else "❌ Not Acceptable"

    # TextBlob correction
    corrected_sentence = get_textblob_correction(sentence)
    correction_needed = "❌ Correction Suggested" if sentence != corrected_sentence else "✔️ No Correction Needed"

    # Output
    print(f"\n🔍 Original: {sentence}")
    print(f"🧠 BERT Prediction: {bert_result} (Confidence: {confidence:.2f})")
    print(f"📝 TextBlob Suggestion: {correction_needed}")
    if sentence != corrected_sentence:
        print(f"➡️ Suggested: {corrected_sentence}")



🔍 Original: hello i ams
🧠 BERT Prediction: ❌ Not Acceptable (Confidence: 0.89)
📝 TextBlob Suggestion: ❌ Correction Suggested
➡️ Suggested: hello i as
